# Tic Tac Toe Singleton Coin On Blockchain Simulator
> Use [singleton_top_layer_v1_1](https://github.com/Chia-Network/chia-blockchain/blob/main/chia/wallet/puzzles/singleton_top_layer_v1_1.py)

<img src="singleton-tic-tac-toe.jpg" alt="Singleton Tic Tac Toe" width="600"/>

In [1]:
%%bash
chia version
cdv --version
python --version

1.5.1.dev0
cdv, version 1.0.8
Python 3.10.5


In [2]:
# chia libraries
from blspy import (PrivateKey, AugSchemeMPL, G2Element)

from chia.consensus.default_constants import DEFAULT_CONSTANTS
from chia.types.blockchain_format.coin import Coin
from chia.types.blockchain_format.program import Program
from chia.types.coin_spend import CoinSpend
from chia.types.condition_opcodes import ConditionOpcode
from chia.types.spend_bundle import SpendBundle
from chia.util.hash import std_hash
from chia.wallet.puzzles import p2_delegated_puzzle_or_hidden_puzzle

from clvm.casts import int_to_bytes
from clvm_tools.clvmc import compile_clvm_text
from clvm_tools.binutils import disassemble

# utils & tic tac toe helper code
import sys
sys.path.insert(0, "../../../shared")
from utils import (load_program, print_program, print_puzzle, print_json, print_push_tx_result)
import singleton_utils

sys.path.insert(0, "./code")
import tic_tac_toe

# load puzzles
tic_tac_toe_puzzle = load_program("./code/tic-tac-toe.clsp", ["./code", "../../../shared"])
coin_puzzle = load_program("./code/coin.clsp", ["./code", "../../../shared"])
terminate_puzzle = load_program("./code/terminate-game.clsp", ["./code", "../../../shared"])

## 1. Set Up A Blockchain Simulator

In [3]:
from cdv.test import Network, Wallet

network: Network = await Network.create()

# use as function parameters
get_coin_records_by_parent_ids = network.sim_client.get_coin_records_by_parent_ids
get_coin_records_by_puzzle_hash = network.sim_client.get_coin_records_by_puzzle_hash
get_coin_record_by_name = network.sim_client.get_coin_record_by_name
get_puzzle_and_solution = network.sim_client.get_puzzle_and_solution        
get_block_records = network.sim_client.get_block_records
get_additions_and_removals = network.sim_client.get_additions_and_removals

await network.farm_block()

alice: Wallet = network.make_wallet("alice")
bob: Wallet = network.make_wallet("bob")
await network.farm_block(farmer=alice)
await network.farm_block(farmer=bob)

print(f'alice balance:\t\t{alice.balance()}')
print(f'alice pk:\t\t{alice.pk()}')
print(f'alice puzzle hash:\t{alice.puzzle_hash}')
print(f'bob balance:\t\t{bob.balance()}')
print(f'alice pk:\t\t{alice.pk()}')
print(f'bob puzzle hash:\t{bob.puzzle_hash}')

# prepare players info
player_one_info = Program.to([alice.pk(), alice.puzzle_hash])
player_two_info = Program.to([bob.pk(), bob.puzzle_hash])

alice balance:		2000000000000
alice pk:		aba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8
alice puzzle hash:	4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3
bob balance:		2000000000000
alice pk:		aba7ed288dd79189bec34698a3437fa7a45f801596d397a4f70081a0956dcdbe998388bfa758f4d49fa421ce3850a6d8
bob puzzle hash:	87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5


## 2. Prepare the Tic Tac Toe Coin (Inner) Puzzle for Singleton

In [4]:
# both players put one XCH
player_amount = 1_000_000_000_000
player_fee = 50_000_000
p2_amount = player_amount - player_fee
game_amount = (player_amount * 2) + 1 # odd amount for singleton

# prepare terminate-game puzzle
# (mod (IS_SINGLETON PLAYER_ONE_HASH PLAYER_TWO_HASH P2_AMOUNT play_result)
curried_terminate_puzzle = terminate_puzzle.curry(
    1, # Terminate Singleton Coin
    alice.puzzle_hash,
    bob.puzzle_hash,
    p2_amount
)

def get_coin_puzzle(board, player):
    # (mod (BOARD V pos)
    curried_tic_tac_toe_puzzle = tic_tac_toe_puzzle.curry(
            Program.to(board), 
            Program.to(player)
        ) 

    #(mod (MOD PLAYER_ONE_INFO PLAYER_TWO_INFO CURRIED_TIC_TAC_TOE_PUZZLE amount position)
    curried_coin_puzzle = coin_puzzle.curry(
        coin_puzzle,
        curried_terminate_puzzle,
        player_one_info,
        player_two_info,
        curried_tic_tac_toe_puzzle,
        game_amount)
    return curried_coin_puzzle

curried_coin_puzzle = get_coin_puzzle([' '] * 9, 'x')

## 3. Prepare Alice, Bob, and Launcher CoinSpends

In [5]:
alice_coin_wrapper = await alice.choose_coin(1_750_000_000_000)
alice_coin = alice_coin_wrapper.as_coin()
alice_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(alice.pk())

In [6]:
from chia.types.blockchain_format.coin import Coin
from chia.types.coin_spend import CoinSpend
from chia.wallet.puzzles import singleton_top_layer_v1_1

# prepare launcher coin
launcher_coin = Coin(
    alice_coin.name(), # alice's coin spend creates the launcher coin
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH, 
    game_amount
)
launcher_id = launcher_coin.name()
print(f'\nlauncher id: {launcher_id}')

singleton_struct = (
    singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
    (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
)

singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
    singleton_struct,
    curried_coin_puzzle, # tic tac toe coin puzzle
)

launcher_solution = Program.to(
    [
        singleton_puzzle.get_tree_hash(),
        game_amount,
        [
            ("game", "tic tac toe"), 
            ("player one", alice.puzzle_hash),
            ("player two", bob.puzzle_hash),
        ]
    ]
)
launcher_announcement = launcher_solution.get_tree_hash()

launcher_coin_spend = CoinSpend(
    launcher_coin,
    singleton_top_layer_v1_1.SINGLETON_LAUNCHER,
    launcher_solution
)
print_json(launcher_coin_spend.to_json_dict())


launcher id: dc49681391711e259f98d39b9d9f9f80d9d0829427a758ff543fb20d7eb2fce8
{
    "coin": {
        "amount": 2000000000001,
        "parent_coin_info": "0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba",
        "puzzle_hash": "0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9"
    },
    "puzzle_reveal": "0xff02ffff01ff04ffff04ff04ffff04ff05ffff04ff0bff80808080ffff04ffff04ff0affff04ffff02ff0effff04ff02ffff04ffff04ff05ffff04ff0bffff04ff17ff80808080ff80808080ff808080ff808080ffff04ffff01ff33ff3cff02ffff03ffff07ff0580ffff01ff0bffff0102ffff02ff0effff04ff02ffff04ff09ff80808080ffff02ff0effff04ff02ffff04ff0dff8080808080ffff01ff0bffff0101ff058080ff0180ff018080",
    "solution": "0xffa06c200b7ce3a0614a1959fd08fdc5cfa72b936db6373abcc90b63d6c54b7ba65aff8601d1a94a2001ffffff8467616d658b7469632074616320746f65ffff8a706c61796572206f6e65a04f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3ffff8a706c617965722074776fa087908e3f85bf4b55c7e7709915c2ce97a1e

In [7]:
# alice's coin creates a launcher coin
alice_conditions = [
    # create launcher coin with the odd_amount (odd)
    Program.to(
        [
            ConditionOpcode.CREATE_COIN,
            singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH,
            game_amount,
        ]),
    # assert launcher coin announcement
    Program.to(
        [
            ConditionOpcode.ASSERT_COIN_ANNOUNCEMENT, 
            std_hash(launcher_id + launcher_announcement)
        ]),
    
    # change
    [ConditionOpcode.CREATE_COIN, alice.puzzle_hash, alice_coin.amount - (player_amount + 1)]
]

alice_delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_conditions(alice_conditions)
alice_delegated_puzzle_solution = p2_delegated_puzzle_or_hidden_puzzle.solution_for_conditions(alice_conditions)
alice_coin_spend = CoinSpend(
    alice_coin,
    alice_puzzle,
    alice_delegated_puzzle_solution
)

alice_coin_message = (
    alice_delegated_puzzle.get_tree_hash()
    + alice_coin.name()
    + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
)
alice_synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    alice.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)
alice_signature: G2Element = AugSchemeMPL.sign(
    alice_synthetic_sk,
    alice_coin_message
)

In [8]:
bob_coin_wrapper = await bob.choose_coin(1_750_000_000_000)
bob_coin = bob_coin_wrapper.as_coin()
bob_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_pk(bob.pk())
# bob's coin create change back to himself
bob_conditions = [
    [ConditionOpcode.CREATE_COIN, bob.puzzle_hash, bob_coin.amount - player_amount],
]

bob_delegated_puzzle = p2_delegated_puzzle_or_hidden_puzzle.puzzle_for_conditions(bob_conditions)
bob_delegated_puzzle_solution = p2_delegated_puzzle_or_hidden_puzzle.solution_for_conditions(bob_conditions)
bob_coin_spend = CoinSpend(
    bob_coin,
    bob_puzzle,
    bob_delegated_puzzle_solution
)

bob_coin_message = (
    bob_delegated_puzzle.get_tree_hash()
    + bob_coin.name()
    + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
)
bob_synthetic_sk: PrivateKey = p2_delegated_puzzle_or_hidden_puzzle.calculate_synthetic_secret_key(
    bob.sk_,
    p2_delegated_puzzle_or_hidden_puzzle.DEFAULT_HIDDEN_PUZZLE_HASH
)
bob_signature: G2Element = AugSchemeMPL.sign(
    bob_synthetic_sk,
    bob_coin_message
)

## 4. Prepare and Push the Spend Bundle

In [9]:
from chia.types.spend_bundle import SpendBundle

agg_sig = AugSchemeMPL.aggregate([alice_signature, bob_signature])
spend_bundle = SpendBundle(
    [
        alice_coin_spend,
        bob_coin_spend,
        launcher_coin_spend
    ],
    agg_sig
)
result = await network.push_tx(spend_bundle)
print_push_tx_result(result)

additions:
{'amount': 749999999999,
 'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
 'puzzle_hash': '0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3'}
{'amount': 2000000000001,
 'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
 'puzzle_hash': '0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9'}
{'amount': 750000000000,
 'parent_coin_info': '0x17ca02c0a209d7e1a3869442ba13ef9468181c4b095b8823aeaf3c27f8e58c34',
 'puzzle_hash': '0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5'}
{'amount': 2000000000001,
 'parent_coin_info': '0xdc49681391711e259f98d39b9d9f9f80d9d0829427a758ff543fb20d7eb2fce8',
 'puzzle_hash': '0x6c200b7ce3a0614a1959fd08fdc5cfa72b936db6373abcc90b63d6c54b7ba65a'}
removals:
{'amount': 1750000000000,
 'parent_coin_info': '0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001',
 'puzzle_hash': '0x4f45877796d7a64e192bcc9f899afeeda

> The launcher coin is created and spent in the same block (both additions and removals):
```
{'amount': 2000000000001,
 'parent_coin_info': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba',
 'puzzle_hash': '0xeff07522495060c066f66f32acc2a77e3a3e737aca8baea4d1a64ea4cdc13da9'}
```

> The first singleton coin
```
{'amount': 2000000000001,
 'parent_coin_info': '0xdc49681391711e259f98d39b9d9f9f80d9d0829427a758ff543fb20d7eb2fce8',
 'puzzle_hash': '0x6c200b7ce3a0614a1959fd08fdc5cfa72b936db6373abcc90b63d6c54b7ba65a'}
```

> Alice's and Bob's Coins that were Spent
```
{'amount': 1750000000000,
 'parent_coin_info': '0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000001',
 'puzzle_hash': '0x4f45877796d7a64e192bcc9f899afeedae391f71af3afd7e15a0792c049d23d3'}
{'amount': 1750000000000,
 'parent_coin_info': '0xe3b0c44298fc1c149afbf4c8996fb92400000000000000000000000000000002',
 'puzzle_hash': '0x87908e3f85bf4b55c7e7709915c2ce97a1e6ec1d227e54a04dbfee6862d546a5'}
```

In [10]:
singleton_coin_puzzle_hash = bytes.fromhex("6c200b7ce3a0614a1959fd08fdc5cfa72b936db6373abcc90b63d6c54b7ba65a")
coin_records = await get_coin_records_by_puzzle_hash(singleton_coin_puzzle_hash)
singleton_tic_tac_toe_coin = coin_records[0].coin
print(singleton_tic_tac_toe_coin)


{'amount': 2000000000001,
 'parent_coin_info': '0xdc49681391711e259f98d39b9d9f9f80d9d0829427a758ff543fb20d7eb2fce8',
 'puzzle_hash': '0x6c200b7ce3a0614a1959fd08fdc5cfa72b936db6373abcc90b63d6c54b7ba65a'}


## 5. Spend Singleton Coin

In [11]:
async def sim_play(prev_coin_spend, singleton_tic_tac_toe_coin, sk, board, player, position):
    network.sim.pass_blocks(1)

    lineage_proof = singleton_top_layer_v1_1.lineage_proof_for_coinsol(prev_coin_spend)
 
    curried_coin_puzzle = get_coin_puzzle(board, player)
    singleton_struct = (
        singleton_top_layer_v1_1.SINGLETON_MOD_HASH, 
        (launcher_id, singleton_top_layer_v1_1.SINGLETON_LAUNCHER_HASH)
    )

    singleton_puzzle = singleton_top_layer_v1_1.SINGLETON_MOD.curry(
                singleton_struct,
                curried_coin_puzzle,
    )

    singleton_coin = await singleton_utils.get_unspent_singleton(
        get_coin_records_by_parent_ids, 
        launcher_id)
    
    curried_tic_tac_toe_puzzle = tic_tac_toe.get_curried_puzzle_from_curried_coin_puzzle(curried_coin_puzzle)
    board_state, new_board = tic_tac_toe.play(curried_tic_tac_toe_puzzle, position)
    tic_tac_toe.print_board(new_board)

    coin_message = (
        std_hash(int_to_bytes(position))
        + singleton_tic_tac_toe_coin.name()
        + DEFAULT_CONSTANTS.AGG_SIG_ME_ADDITIONAL_DATA
    )

    # sign with sk
    signature: G2Element = AugSchemeMPL.sign(
        sk,
        coin_message
    )
    
    inner_solution = Program.to([position]) # position
    coin_spend = singleton_utils.get_singleton_coin_spend(
        singleton_coin, singleton_puzzle, lineage_proof,
        inner_solution
    )

    print(f'singleton_coin:\n{singleton_coin}')
    print(f'\nlineage_proof:\n{lineage_proof}')
    print(f'\ninner_solution:\n{inner_solution}')
    print(f'\nsingleton_puzzle:\n{singleton_puzzle.get_tree_hash()}')

    print('\ncoin_spend:')
    print_json(coin_spend.to_json_dict())

    spend_bundle = SpendBundle([coin_spend], signature)
    #print_json(spend_bundle.to_json_dict())
    result = await network.push_tx(spend_bundle)
    print(result)
    singleton_tic_tac_toe_coin = result["additions"][0]
    print(singleton_tic_tac_toe_coin)
    next_player = 'x' if player == 'o' else 'o'
    return new_board, next_player, singleton_tic_tac_toe_coin, coin_spend

In [12]:
board = [' '] * 9
player = 'x'
board, player, singleton_tic_tac_toe_coin, coin_spend = await sim_play(launcher_coin_spend, singleton_tic_tac_toe_coin, alice.sk_, board, player, 4)
board, player, singleton_tic_tac_toe_coin, coin_spend = await sim_play(launcher_coin_spend, singleton_tic_tac_toe_coin, bob.sk_, board, player, 1)




   |   |   
---+---+---
   | x |   
---+---+---
   |   |   

singleton_coin:
{'amount': 2000000000001,
 'parent_coin_info': '0xdc49681391711e259f98d39b9d9f9f80d9d0829427a758ff543fb20d7eb2fce8',
 'puzzle_hash': '0x6c200b7ce3a0614a1959fd08fdc5cfa72b936db6373abcc90b63d6c54b7ba65a'}

lineage_proof:
{'amount': 2000000000001,
 'inner_puzzle_hash': None,
 'parent_name': '0x12d7b8c1654f82f2330059abc28e3240e863450706de7fdc518026f393f68bba'}

inner_solution:
ff0480

singleton_puzzle:
6c200b7ce3a0614a1959fd08fdc5cfa72b936db6373abcc90b63d6c54b7ba65a

coin_spend:
{
    "coin": {
        "amount": 2000000000001,
        "parent_coin_info": "0xdc49681391711e259f98d39b9d9f9f80d9d0829427a758ff543fb20d7eb2fce8",
        "puzzle_hash": "0x6c200b7ce3a0614a1959fd08fdc5cfa72b936db6373abcc90b63d6c54b7ba65a"
    },
    "puzzle_reveal": "0xff02ffff01ff02ffff01ff02ffff03ffff18ff2fff3480ffff01ff04ffff04ff20ffff04ff2fff808080ffff04ffff02ff3effff04ff02ffff04ff05ffff04ffff02ff2affff04ff02ffff04ff27ffff04ffff02ffff0

KeyError: 'additions'